In [1]:
import sys
import os
import h5py
from collections import Counter
from progressbar import *
import re
import numpy as np
import h5py
from scipy import signal
import matplotlib
from repeating_classifier_training_utils import *
from functools import reduce
from tqdm import tqdm

# Add the path to the parent directory to augment search for module
par_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

if par_dir not in sys.path:
    sys.path.append(par_dir)
%load_ext autoreload
%matplotlib inline
%autoreload 2
index_path = os.path.join(os.getcwd(), 'Index_Storage')

In [2]:
event_data_file = h5py.File('/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_full_tank_pointnet.h5','r')
dset_labels = np.array(event_data_file['labels'])

In [3]:
idx_file = np.load('/fast_scratch/WatChMaL/data/IWCD_fulltank_300_pe_idxs.npz', allow_pickle=True)
test_idxs = idx_file['test_idxs'].astype(int)
train_idxs = idx_file['train_idxs'].astype(int)
val_idxs = idx_file['val_idxs'].astype(int)

In [4]:
test_labels = dset_labels[test_idxs]
train_labels = dset_labels[train_idxs]
val_labels = dset_labels[val_idxs]

In [5]:
no_muon_train_idxs = train_idxs[np.where(train_labels!=2)[0]]
no_muon_val_idxs = val_idxs[np.where(val_labels!=2)[0]]
no_muon_test_idxs = test_idxs[np.where(test_labels!=2)[0]]

In [6]:
np.savez('/fast_scratch/WatChMaL/data/IWCD_fulltank_300_pe_idxs_no_muons.npz', train_idxs=no_muon_train_idxs,test_idxs=no_muon_test_idxs,val_idxs=no_muon_val_idxs)

## Check

In [3]:
no_muon_idx_file = np.load('/fast_scratch/WatChMaL/data/IWCD_fulltank_300_pe_idxs_no_muons.npz', allow_pickle=True)
nm_train_idxs = no_muon_idx_file['train_idxs']
nm_val_idxs = no_muon_idx_file['val_idxs']
nm_test_idxs = no_muon_idx_file['test_idxs']

In [8]:
print(f"Training Set: {nm_train_idxs.shape} Val Set: {nm_val_idxs.shape} Test Set: {nm_test_idxs.shape}")

Training Set: (2813146,) Val Set: (563020,) Test Set: (2251726,)


In [9]:
idx_file = np.load('/fast_scratch/WatChMaL/data/IWCD_fulltank_300_pe_idxs.npz', allow_pickle=True)
test_idxs = idx_file['test_idxs'].astype(int)
train_idxs = idx_file['train_idxs'].astype(int)
val_idxs = idx_file['val_idxs'].astype(int)

In [11]:
print(f"Training Set: {train_idxs.shape} Val Set: {val_idxs.shape} Test Set: {test_idxs.shape}")

Training Set: (4188774,) Val Set: (837754,) Test Set: (3351020,)


In [13]:
event_data_file = h5py.File('/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_full_tank_pointnet.h5','r')
dset_labels = np.array(event_data_file['labels'])
event_data_file.close()

In [15]:
for idx_set in [nm_train_idxs, nm_val_idxs, nm_test_idxs]:
    for idx in tqdm(idx_set):
        assert dset_labels[idx] != 2

100%|██████████| 2251726/2251726 [00:06<00:00, 367506.30it/s]
